In [6]:
import ee
ee.Authenticate()  
ee.Initialize()

In [12]:
import ee
import geemap
from datetime import datetime, timedelta

# Inisialisasi EE
ee.Initialize()

# Area Jakarta
jakarta = ee.Geometry.Rectangle([106.6, -6.4, 107.0, -6.1])

# Cari citra Sentinel-1 terbaru
def get_latest_sentinel1_vv(region, days_back=60):
    today = datetime.utcnow()
    start_date = today - timedelta(days=days_back)

    collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                  .filterBounds(region)
                  .filterDate(start_date.strftime('%Y-%m-%d'), today.strftime('%Y-%m-%d'))
                  .filter(ee.Filter.eq('instrumentMode', 'IW'))
                  .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
                  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                  .sort('system:time_start', False))  # Urut dari terbaru

    image = collection.first()
    info = image.get('system:time_start').getInfo()
    date_str = datetime.utcfromtimestamp(info / 1000).strftime('%Y-%m-%d')

    return image.select('VV').clip(region), date_str

# Ambil citra terbaru
latest_img, latest_date = get_latest_sentinel1_vv(jakarta)
print("Tanggal citra terbaru:", latest_date)

# Tampilkan di peta
Map = geemap.Map(center=[-6.2, 106.8], zoom=10)
Map.addLayer(latest_img, {'min': -25, 'max': 0}, f'Sentinel-1 VV {latest_date}')
Map.addLayer(jakarta, {}, 'Jakarta AOI', False)
Map


Tanggal citra terbaru: 2025-06-12


Map(center=[-6.2, 106.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…